<a href="https://colab.research.google.com/github/taekjunkim/artiphysiology2020/blob/main/RFsize_fromCNNs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#package for calc rf on architecture
!git clone https://github.com/jotaf98/easy-receptive-fields-pytorch.git
import sys
sys.path.append('/content/easy-receptive-fields-pytorch')
from receptivefield import receptivefield

import torch.nn as nn
import torchvision.models as models

import numpy as np;

Cloning into 'easy-receptive-fields-pytorch'...
remote: Enumerating objects: 24, done.
remote: Total 24 (delta 0), reused 0 (delta 0), pack-reused 24
Unpacking objects: 100% (24/24), done.


In [ ]:
alexnet = models.alexnet(pretrained=False);
#print(alexnet);
layers_of_interest = [0,3,6,8,10];

for i in range(len(layers_of_interest)):
    endLayer = layers_of_interest[i]
    net = alexnet.features[:endLayer+1];
    rfsize = receptivefield(net,(1,3,224,224)).rfsize;
    outputsize = receptivefield(net,(1,3,224,224)).outputsize;    
    stride = receptivefield(net,(1,3,224,224)).stride;        
    print('Conv'+str(i+1)+': '+str(endLayer)+' --> RF size: '
          +str(rfsize)+', '+str(net[-1].out_channels)+' units, Stride: '+str(stride));

Conv1: 0 --> RF size: Size(w=11, h=11), 64 units, Output size: Size(w=55, h=55), Stride: Vector(x=4, y=4)
Conv2: 3 --> RF size: Size(w=51, h=51), 192 units, Output size: Size(w=27, h=27), Stride: Vector(x=8, y=8)
Conv3: 6 --> RF size: Size(w=99, h=99), 384 units, Output size: Size(w=13, h=13), Stride: Vector(x=16, y=16)
Conv4: 8 --> RF size: Size(w=131, h=131), 256 units, Output size: Size(w=13, h=13), Stride: Vector(x=16, y=16)
Conv5: 10 --> RF size: Size(w=163, h=163), 256 units, Output size: Size(w=13, h=13), Stride: Vector(x=16, y=16)


In [ ]:
vgg16 = models.vgg16(pretrained=False);
layers_of_interest = [0,2,5,7,10,12,14,17,19,21,24,26,28];

for i in range(len(layers_of_interest)):
    endLayer = layers_of_interest[i]
    net = vgg16.features[:endLayer+1];
    rfsize = receptivefield(net,(1,3,224,224)).rfsize;
    outputsize = receptivefield(net,(1,3,224,224)).outputsize;    
    stride = receptivefield(net,(1,3,224,224)).stride;        
    print('Conv'+str(i+1)+': '+str(endLayer)+' --> RF size: '
          +str(rfsize)+', '+str(net[-1].out_channels)+' units, Stride: '+str(stride));

Conv1: 0 --> RF size: Size(w=3, h=3), 64 units, Output size: Size(w=224, h=224), Stride: Vector(x=1, y=1)
Conv2: 2 --> RF size: Size(w=5, h=5), 64 units, Output size: Size(w=224, h=224), Stride: Vector(x=1, y=1)
Conv3: 5 --> RF size: Size(w=10, h=10), 128 units, Output size: Size(w=112, h=112), Stride: Vector(x=2, y=2)
Conv4: 7 --> RF size: Size(w=14, h=14), 128 units, Output size: Size(w=112, h=112), Stride: Vector(x=2, y=2)
Conv5: 10 --> RF size: Size(w=24, h=24), 256 units, Output size: Size(w=56, h=56), Stride: Vector(x=4, y=4)
Conv6: 12 --> RF size: Size(w=32, h=32), 256 units, Output size: Size(w=56, h=56), Stride: Vector(x=4, y=4)
Conv7: 14 --> RF size: Size(w=40, h=40), 256 units, Output size: Size(w=56, h=56), Stride: Vector(x=4, y=4)
Conv8: 17 --> RF size: Size(w=60, h=60), 512 units, Output size: Size(w=28, h=28), Stride: Vector(x=8, y=8)
Conv9: 19 --> RF size: Size(w=76, h=76), 512 units, Output size: Size(w=28, h=28), Stride: Vector(x=8, y=8)
Conv10: 21 --> RF size: Size(w

In [ ]:
resnet50 = models.resnet50(pretrained=False);
#print(resnet50)

class MyNet(nn.Module):
    def __init__(self,original_model,endLayer=19):
        super(MyNet,self).__init__();

        stages =  list(original_model.children())[:4] + \
                  list(original_model.layer1) + \
                  list(original_model.layer2) + \
                  list(original_model.layer3) + \
                  list(original_model.layer4);

        self.features = nn.Sequential(*stages[:endLayer+1]);

    def forward(self,x):
        x = self.features(x);
        self.out_channels = np.shape(x)[1]
        return x;

MyResnet = MyNet(resnet50);
MyResnet

MyNet(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (4): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bi

In [ ]:
layers_of_interest = [0,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19];

for i in range(len(layers_of_interest)):
    endLayer = layers_of_interest[i];
    net = MyNet(resnet50,endLayer);

    try:
        rfsize = receptivefield(net,(1,3,224,224)).rfsize;
        outputsize = receptivefield(net,(1,3,224,224)).outputsize;    
        stride = receptivefield(net,(1,3,224,224)).stride;       
    except:
        rfsize = receptivefield(net,(1,3,500,500)).rfsize;
        outputsize = receptivefield(net,(1,3,500,500)).outputsize;    
        stride = receptivefield(net,(1,3,500,500)).stride;        

    if i==0: 
        print('Conv1:'+str(endLayer)+': '+str(endLayer)+' --> RF size: '
              +str(rfsize)+', '+str(net.out_channels)+' units, Stride: '+str(stride));      
    elif net.out_channels==256:
        print('layer1_'+str(endLayer-3)+': '+str(endLayer)+' --> RF size: '
              +str(rfsize)+', '+str(net.out_channels)+' units, Stride: '+str(stride));      
    elif net.out_channels==512:
        print('layer2_'+str(endLayer-6)+': '+str(endLayer)+' --> RF size: '
              +str(rfsize)+', '+str(net.out_channels)+' units, Stride: '+str(stride));      
    elif net.out_channels==1024:
        print('layer3_'+str(endLayer-10)+': '+str(endLayer)+' --> RF size: '
              +str(rfsize)+', '+str(net.out_channels)+' units, Stride: '+str(stride));      
    elif net.out_channels==2048:
        print('layer3_'+str(endLayer-16)+': '+str(endLayer)+' --> RF size: '
              +str(rfsize)+', '+str(net.out_channels)+' units, Stride: '+str(stride));      

Conv1:0: 0 --> RF size: Size(w=7, h=7), 64 units, Stride: Vector(x=2, y=2)
layer1_1: 4 --> RF size: Size(w=19, h=19), 256 units, Stride: Vector(x=4, y=4)
layer1_2: 5 --> RF size: Size(w=27, h=27), 256 units, Stride: Vector(x=4, y=4)
layer1_3: 6 --> RF size: Size(w=35, h=35), 256 units, Stride: Vector(x=4, y=4)
layer2_1: 7 --> RF size: Size(w=43, h=43), 512 units, Stride: Vector(x=8, y=8)
layer2_2: 8 --> RF size: Size(w=59, h=59), 512 units, Stride: Vector(x=8, y=8)
layer2_3: 9 --> RF size: Size(w=75, h=75), 512 units, Stride: Vector(x=8, y=8)
layer2_4: 10 --> RF size: Size(w=91, h=91), 512 units, Stride: Vector(x=8, y=8)
layer3_1: 11 --> RF size: Size(w=107, h=107), 1024 units, Stride: Vector(x=16, y=16)
layer3_2: 12 --> RF size: Size(w=139, h=139), 1024 units, Stride: Vector(x=16, y=16)
layer3_3: 13 --> RF size: Size(w=171, h=171), 1024 units, Stride: Vector(x=16, y=16)
layer3_4: 14 --> RF size: Size(w=198, h=198), 1024 units, Stride: Vector(x=11, y=11)
layer3_5: 15 --> RF size: Size(